In [1]:
import numpy as np
import pandas as pd
from astropy.utils.data import get_pkg_data_filename
from astropy.table import QTable
import math
from synphot import SourceSpectrum, Observation
from synphot import units
from astropy import units as u
import stsynphot as stsyn

In [2]:
class Targets:
    def __init__(self,name,redshift,lum_dist):
        self.n = name
        self.z = redshift
        self.Ld = lum_dist
        
        
s1 = Targets('1025',0.361,1940.5)
#print (source1.n, source1.z, source1.Ld)

s2 = Targets('1037',0.091,417.2)
s3 = Targets('1128',0.404,2217.0)
s4 = Targets('1201',0.445,2488.2)
s5 = Targets('1203',0.371,2004.1)
s6 = Targets('1221',0.171,826.8)

sources = [s1, s2, s3, s4, s5, s6]

#print (sources[1].n)

In [3]:
for s in sources:
    print (s.n, s.z, s.Ld)

1025 0.361 1940.5
1037 0.091 417.2
1128 0.404 2217.0
1201 0.445 2488.2
1203 0.371 2004.1
1221 0.171 826.8


In [4]:
def redshift(model, r):
     
    model_z = SourceSpectrum(sp.model, z=r)
    model_zc = SourceSpectrum(sp.model, z=r, z_type='conserve_flux')
    
    return model_z, model_zc


def convolved_obs(target_name, model_sp, model_sp_cf):
    
    filter_lambda = 0.0*u.AA    # pivot or effective wavelength
    bp_225 = stsyn.band('wfc3,uvis2,f225w')
    bp_275 = stsyn.band('wfc3,uvis2,f275w')  
    bp_336 = stsyn.band('wfc3,uvis2,f336w')

    if target_name == '1037':
        bp = bp_225
        filter_lambda = 2341.0*u.AA
    elif target_name == '1221':
        bp = bp_275
        filter_lambda = 2715.3*u.AA 
    else:
        bp = bp_336
        filter_lambda = 3361.1*u.AA 
    
    obs = Observation(model_sp, bp, binset=bp.binset)
    obs_cf = Observation(model_sp_cf, bp, binset=bp.binset)
    
    return filter_lambda, obs, obs_cf



In [5]:
#   CHOOSING only one age/epoch to compute flux of --> 10^7 yrs

#   read in entire table and use 'names' to select two speicifc columns       OR   
#   use 'usecols' with read_table to select only columns 0 and 10 (wavelength & 10Myr)    


c = 3.0e18*(u.AA/u.s) 


In [6]:
#                          ********** Continuous **********

inputdir_cont = '/Users/orion/phd_research/stellar_mod/s99_models/continuous/'

In [7]:
for s in sources:
    
    print('\n\nSource:', s.n)
    
    for n in range(2,14,2):

        df = pd.read_table(inputdir_cont+'fig'+str(n)+'c.dat', sep='\s+', skiprows=[0,1,2], usecols=[0,10],
                           names=['Wavelength','10Myr'])
    
        print('\nModel: fig'+str(n)+'c.dat')
    
        f_lambda = np.zeros(2, dtype=np.float32)
        f_nu = np.zeros(2, dtype=np.float32)
        ab_mag = np.zeros(2, dtype=np.float32)
    
        d = s.Ld * 3.08e24
    
        df['new_col'] = 10**df['10Myr']
        df['flux_col'] = df['new_col'] / (4 * math.pi * d**2)
    
        df.to_csv('buffer_new.txt', columns=['Wavelength','flux_col'], sep='\t',
                    header=False, index=False) 

        #print (df['flux_col'])
        sp = SourceSpectrum.from_file('buffer_new.txt')
    
        model_z, model_zcf = redshift(sp,s.z)
        filter_efflambda, obs_z, obs_zcf = convolved_obs(s.n, model_z, model_zcf) 
    
        f_lambda = [obs_z.effstim('flam'), obs_zcf.effstim('flam')] 
        f_nu = [k * (filter_efflambda**2 / c) for k in f_lambda]
    
        ab_mag = [round((- 2.5 * (math.log10(k/units.FNU)) - 48.6), 2) for k in f_nu]
    
        print('AB mag (z) for 10 Myr epoch :', ab_mag[0], ',\t', f_lambda[0])
        print('AB mag (zcf) for 10 Myr epoch :', ab_mag[1], ',\t',f_lambda[1])
        
        if n==2:
            rows=[]
            #rows.append(ab_mag) 
            rows.append(f_lambda)
        else: 
            #rows.append(ab_mag)
            rows.append(f_lambda)
    
        t = QTable(rows=rows, names=('z','z_cf'))
    
    t.write(str(s.n)+'_fluxes_cont.txt', format='ascii', delimiter='\t', overwrite=True)





Source: 1025

Model: fig2c.dat
AB mag (z) for 10 Myr epoch : 22.75 ,	 7.647403473823824e-18 FLAM
AB mag (zcf) for 10 Myr epoch : 23.09 ,	 5.618959201927864e-18 FLAM

Model: fig4c.dat
AB mag (z) for 10 Myr epoch : 24.76 ,	 1.1975170183302664e-18 FLAM
AB mag (zcf) for 10 Myr epoch : 25.1 ,	 8.798802485894686e-19 FLAM

Model: fig6c.dat
AB mag (z) for 10 Myr epoch : 23.39 ,	 4.254227407864949e-18 FLAM
AB mag (zcf) for 10 Myr epoch : 23.72 ,	 3.125809998431263e-18 FLAM

Model: fig8c.dat
AB mag (z) for 10 Myr epoch : 22.9 ,	 6.652353909378046e-18 FLAM
AB mag (zcf) for 10 Myr epoch : 23.24 ,	 4.887842696089673e-18 FLAM

Model: fig10c.dat
AB mag (z) for 10 Myr epoch : 24.84 ,	 1.1135286767117706e-18 FLAM
AB mag (zcf) for 10 Myr epoch : 25.18 ,	 8.181694906037991e-19 FLAM

Model: fig12c.dat
AB mag (z) for 10 Myr epoch : 23.45 ,	 4.027845560619916e-18 FLAM
AB mag (zcf) for 10 Myr epoch : 23.78 ,	 2.959475062909563e-18 FLAM


Source: 1037

Model: fig2c.dat
AB mag (z) for 10 Myr epoch : 19.57 ,	

In [19]:
#                          ********** Instantaneous **********

inputdir_inst = '/Users/orion/phd_research/stellar_mod/s99_models/instantaneous/'

In [20]:
for s in sources:
    
    print('\n\nSource:', s.n)
    
    for n in range(1,13,2):

        df = pd.read_table(inputdir_inst+'fig'+str(n)+'c.dat', sep='\s+', skiprows=[0,1,2], usecols=[0,10],
                           names=['Wavelength','10Myr'])
    
        print('\nModel: fig'+str(n)+'c.dat')
    
        f_lambda = np.zeros(2, dtype=np.float32)
        f_nu = np.zeros(2, dtype=np.float32)
        ab_mag = np.zeros(2, dtype=np.float32)
    
        d = s.Ld * 3.08e24
    
        df['new_col'] = 10**df['10Myr']
        df['flux_col'] = df['new_col'] / (4 * math.pi * d**2)
    
        df.to_csv('buffer_new.txt', columns=['Wavelength','flux_col'], sep='\t',
                    header=False, index=False) 

        #print (df['flux_col'])
        sp = SourceSpectrum.from_file('buffer_new.txt')
    
        model_z, model_zcf = redshift(sp,s.z)
        filter_efflambda, obs_z, obs_zcf = convolved_obs(s.n, model_z, model_zcf) 
    
        f_lambda = [obs_z.effstim('flam'), obs_zcf.effstim('flam')] 
        f_nu = [k * (filter_efflambda**2 / c) for k in f_lambda]
    
        ab_mag = [round((- 2.5 * (math.log10(k/units.FNU)) - 48.6), 2) for k in f_nu]
    
        print('AB mag (z) for 10 Myr epoch :', ab_mag[0], ',\t', f_lambda[0])
        print('AB mag (zcf) for 10 Myr epoch :', ab_mag[1], ',\t',f_lambda[1])
        
        if n==1:
            rows=[]
            #rows.append(ab_mag)
            rows.append(f_lambda)
        else: 
            #rows.append(ab_mag)
            rows.append(f_lambda)
    
        t = QTable(rows=rows, names=('z','z_cf'))
    
    t.write(str(s.n)+'_fluxes_instt.txt', format='ascii', delimiter='\t', overwrite=True)





Source: 1025

Model: fig1c.dat
AB mag (z) for 10 Myr epoch : 26.44 ,	 2.5706255544579884e-19 FLAM
AB mag (zcf) for 10 Myr epoch : 26.77 ,	 1.888777042217479e-19 FLAM

Model: fig3c.dat
AB mag (z) for 10 Myr epoch : 27.7 ,	 7.98505126443652e-20 FLAM
AB mag (zcf) for 10 Myr epoch : 28.04 ,	 5.86704721854263e-20 FLAM

Model: fig5c.dat
AB mag (z) for 10 Myr epoch : 26.28 ,	 2.956893028979969e-19 FLAM
AB mag (zcf) for 10 Myr epoch : 26.62 ,	 2.172588559132968e-19 FLAM

Model: fig7c.dat
AB mag (z) for 10 Myr epoch : 26.44 ,	 2.5579173371577357e-19 FLAM
AB mag (zcf) for 10 Myr epoch : 26.78 ,	 1.8794396305347063e-19 FLAM

Model: fig9c.dat
AB mag (z) for 10 Myr epoch : 27.71 ,	 7.955057165129346e-20 FLAM
AB mag (zcf) for 10 Myr epoch : 28.04 ,	 5.845008938375714e-20 FLAM

Model: fig11c.dat
AB mag (z) for 10 Myr epoch : 26.29 ,	 2.942276293878741e-19 FLAM
AB mag (zcf) for 10 Myr epoch : 26.62 ,	 2.161848856633902e-19 FLAM


Source: 1037

Model: fig1c.dat
AB mag (z) for 10 Myr epoch : 23.26 ,	 